In [1]:
import re
import csv
import nltk
import keras
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
train_set_path = '/kaggle/input/ag-news-sinhala/ag_news_sinhala/train.csv'
test_set_path = '/kaggle/input/ag-news-sinhala/ag_news_sinhala/test.csv'

In [3]:
stop_words = '/kaggle/input/stop-words/stop words.txt'
stem_dictionary = '/kaggle/input/stem-dictionary/stem_dictionary.txt'

In [4]:
train_set = pd.read_csv(train_set_path)
test_set = pd.read_csv(test_set_path)

In [5]:
# Concatenate train and test data
df = pd.concat([train_set, test_set], ignore_index=True)

# Save the combined data to a new CSV file
df.to_csv('combined.csv', index=False)

In [6]:
df.head()

,Unnamed: 0,Class Index,Title (English),Description (English),Title (Sinhala),Description (Sinhala)
0,0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli...",වෝල් වීදිය නැවතත් ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය.,"නිව් යෝර්ක් (රොයිටර්) - කෙටි විකුණුම්කරුවන්, ව..."
1,1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...,කාර්ලයිල් වාණිජ අභ්‍යවකාශය දෙස බලයි (රොයිටර්),රොයිටර් - පුද්ගලික ආයෝජන සමාගමක් වන Carlyle Gr...
2,2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...,තෙල් සහ ආර්ථිකය,රොයිටර් - බොරතෙල් මිල ඉහළ යාම සහ ආර්ථිකය පිළිබ...
3,3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...,ඉරාකය ප්‍රධාන දකුණු නල මාර්ගයෙන් (රොයිටර්) තෙල...,රොයිටර් - කැරලිකාර මිලීෂියාවකට යටිතල පහසුකම්වල...
4,4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco...",එක්සත් ජනපද ආර්ථිකයට (AFP) නව තර්ජනයක් එල්ල කර...,"AFP - ඉරාක ලෝක තෙල් මිල, වාර්තා පෙරලීම සහ මුදල..."


In [7]:
df = df.drop(['Unnamed: 0', 'Title (English)', 'Description (English)'], axis=1).copy()

In [8]:
df.head()

,Class Index,Title (Sinhala),Description (Sinhala)
0,3,වෝල් වීදිය නැවතත් ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය.,"නිව් යෝර්ක් (රොයිටර්) - කෙටි විකුණුම්කරුවන්, ව..."
1,3,කාර්ලයිල් වාණිජ අභ්‍යවකාශය දෙස බලයි (රොයිටර්),රොයිටර් - පුද්ගලික ආයෝජන සමාගමක් වන Carlyle Gr...
2,3,තෙල් සහ ආර්ථිකය,රොයිටර් - බොරතෙල් මිල ඉහළ යාම සහ ආර්ථිකය පිළිබ...
3,3,ඉරාකය ප්‍රධාන දකුණු නල මාර්ගයෙන් (රොයිටර්) තෙල...,රොයිටර් - කැරලිකාර මිලීෂියාවකට යටිතල පහසුකම්වල...
4,3,එක්සත් ජනපද ආර්ථිකයට (AFP) නව තර්ජනයක් එල්ල කර...,"AFP - ඉරාක ලෝක තෙල් මිල, වාර්තා පෙරලීම සහ මුදල..."


## Combining Title and Description into one paragraph

In [9]:
df['Paragraph'] = df['Title (Sinhala)'] + " " + df['Description (Sinhala)']

df.head()

,Class Index,Title (Sinhala),Description (Sinhala),Paragraph
0,3,වෝල් වීදිය නැවතත් ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය.,"නිව් යෝර්ක් (රොයිටර්) - කෙටි විකුණුම්කරුවන්, ව...",වෝල් වීදිය නැවතත් ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය...
1,3,කාර්ලයිල් වාණිජ අභ්‍යවකාශය දෙස බලයි (රොයිටර්),රොයිටර් - පුද්ගලික ආයෝජන සමාගමක් වන Carlyle Gr...,කාර්ලයිල් වාණිජ අභ්‍යවකාශය දෙස බලයි (රොයිටර්) ...
2,3,තෙල් සහ ආර්ථිකය,රොයිටර් - බොරතෙල් මිල ඉහළ යාම සහ ආර්ථිකය පිළිබ...,තෙල් සහ ආර්ථිකය රොයිටර් - බොරතෙල් මිල ඉහළ යාම ...
3,3,ඉරාකය ප්‍රධාන දකුණු නල මාර්ගයෙන් (රොයිටර්) තෙල...,රොයිටර් - කැරලිකාර මිලීෂියාවකට යටිතල පහසුකම්වල...,ඉරාකය ප්‍රධාන දකුණු නල මාර්ගයෙන් (රොයිටර්) තෙල...
4,3,එක්සත් ජනපද ආර්ථිකයට (AFP) නව තර්ජනයක් එල්ල කර...,"AFP - ඉරාක ලෝක තෙල් මිල, වාර්තා පෙරලීම සහ මුදල...",එක්සත් ජනපද ආර්ථිකයට (AFP) නව තර්ජනයක් එල්ල කර...


In [10]:
df.drop(df.columns[[1, 2]], axis=1, inplace=True)
df.head()

,Class Index,Paragraph
0,3,වෝල් වීදිය නැවතත් ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය...
1,3,කාර්ලයිල් වාණිජ අභ්‍යවකාශය දෙස බලයි (රොයිටර්) ...
2,3,තෙල් සහ ආර්ථිකය රොයිටර් - බොරතෙල් මිල ඉහළ යාම ...
3,3,ඉරාකය ප්‍රධාන දකුණු නල මාර්ගයෙන් (රොයිටර්) තෙල...
4,3,එක්සත් ජනපද ආර්ථිකයට (AFP) නව තර්ජනයක් එල්ල කර...


In [11]:
df.to_csv('new_df.csv', index=False)

In [12]:
dataframe = '/kaggle/input/sinhala-paragraphs/Sinhala_Paragraphs.csv'

In [13]:
df = pd.read_csv(dataframe)
df.head()

,Class Index,Paragraph
0,3,වෝල් වීදිය නැවතත් ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය...
1,3,කාර්ලයිල් වාණිජ අභ්‍යවකාශය දෙස බලයි (රොයිටර්) ...
2,3,තෙල් සහ ආර්ථිකය රොයිටර් - බොරතෙල් මිල ඉහළ යාම ...
3,3,ඉරාකය ප්‍රධාන දකුණු නල මාර්ගයෙන් (රොයිටර්) තෙල...
4,3,එක්සත් ජනපද ආර්ථිකයට (AFP) නව තර්ජනයක් එල්ල කර...


In [14]:
df.shape

(127600, 2)

In [15]:
df['Class Index'].value_counts()

3    31900
4    31900
2    31900
1    31900
Name: Class Index, dtype: int64

## Cleaning paragraph (Tokenizing, Removing punctuations, Removing non-Latin characters)

In [16]:
def clean_text(text):
    tokens = nltk.word_tokenize(text)
    regex = re.compile(u'[^\u0D80-\u0DFF]', re.UNICODE)
    tokens = [regex.sub('', w) for w in tokens]
    tokens = [c for c in tokens if c not in string.punctuation]
    return tokens

In [17]:
df['Cleaned Paragraph'] = df['Paragraph'].apply(lambda x: clean_text(x))
df.head()

,Class Index,Paragraph,Cleaned Paragraph
0,3,වෝල් වීදිය නැවතත් ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය...,"[වෝල්, වීදිය, නැවතත්, ධනාත්මක, ආර්ථිකයක්, ලබා,..."
1,3,කාර්ලයිල් වාණිජ අභ්‍යවකාශය දෙස බලයි (රොයිටර්) ...,"[කාර්ලයිල්, වාණිජ, අභ්යවකාශය, දෙස, බලයි, රොයිට..."
2,3,තෙල් සහ ආර්ථිකය රොයිටර් - බොරතෙල් මිල ඉහළ යාම ...,"[තෙල්, සහ, ආර්ථිකය, රොයිටර්, බොරතෙල්, මිල, ඉහළ..."
3,3,ඉරාකය ප්‍රධාන දකුණු නල මාර්ගයෙන් (රොයිටර්) තෙල...,"[ඉරාකය, ප්රධාන, දකුණු, නල, මාර්ගයෙන්, රොයිටර්,..."
4,3,එක්සත් ජනපද ආර්ථිකයට (AFP) නව තර්ජනයක් එල්ල කර...,"[එක්සත්, ජනපද, ආර්ථිකයට, නව, තර්ජනයක්, එල්ල, ක..."


## Removing Stopwords

In [18]:
stopwordsreader = open(stop_words, 'r', encoding='utf-8')
reader = csv.reader(stopwordsreader)

swords = []
for row in reader:
    swords.append(row[0])
print(swords)

['සහ', 'සමග', 'සමඟ', 'අහා', 'ආහ්', 'ආ', 'ඕහෝ', 'අනේ', 'අඳෝ', 'අපොයි', 'අපෝ', 'අයියෝ', 'ආයි', 'ඌයි', 'චී', 'චිහ්', 'චික්', 'හෝ\u200d', 'දෝ', 'දෝහෝ', 'මෙන්', 'සේ', 'වැනි', 'බඳු', 'වන්', 'අයුරු', 'අයුරින්', 'ලෙස', 'වැඩි', 'ශ්\u200dරී', 'හා', 'ය', 'නිසා', 'නිසාවෙන්', 'බවට', 'බව', 'බවෙන්', 'නම්', 'වැඩි', 'සිට', 'දී', 'මහා', 'මහ', 'පමණ', 'පමණින්', 'පමන', 'වන', 'විට', 'විටින්', 'මේ', 'මෙලෙස', 'මෙයින්', 'ඇති', 'ලෙස', 'සිදු', 'වශයෙන්', 'යන', 'සඳහා', 'මගින්', 'හෝ\u200d', 'ඉතා', 'ඒ', 'එම', 'ද', 'අතර', 'විසින්', 'සමග', 'පිළිබඳව', 'පිළිබඳ', 'තුළ', 'බව', 'වැනි', 'මහ', 'මෙම', 'මෙහි', 'මේ', 'වෙත', 'වෙතින්', 'වෙතට', 'වෙනුවෙන්', 'වෙනුවට', 'වෙන', 'ගැන', 'නෑ', 'අනුව', 'නව', 'පිළිබඳ', 'විශේෂ', 'දැනට', 'එහෙන්', 'මෙහෙන්', 'එහේ', 'මෙහේ', 'ම', 'තවත්', 'තව ', 'සහ', 'දක්වා', 'ට', 'ගේ', 'එ', 'ක', 'ක්', 'බවත්', 'බවද', 'මත', 'ඇතුලු', 'ඇතුළු', 'මෙසේ', 'වඩා', 'වඩාත්ම', 'නිති', 'නිතිත්', 'නිතොර', 'නිතර', 'ඉක්බිති', 'දැන්', 'යලි', 'පුන', 'ඉතින්', 'සිට', 'සිටන්', 'පටන්', 'තෙක්', 'දක්වා', 'සා', 'තාක්', 'තුවක්', 'පවා', 'ද

In [19]:
def remove_stop_words(tokens):
    regex = re.compile(u'[^\u0D80-\u0DFF]', re.UNICODE)
    tokens = [regex.sub('', w) for w in tokens]
    tokens = [c for c in tokens if c not in string.punctuation]
    return tokens

In [20]:
df['Stopwords Removed (Paragraph)'] = df['Cleaned Paragraph'].apply(lambda x: remove_stop_words(x))
df.head()

,Class Index,Paragraph,Cleaned Paragraph,Stopwords Removed (Paragraph)
0,3,වෝල් වීදිය නැවතත් ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය...,"[වෝල්, වීදිය, නැවතත්, ධනාත්මක, ආර්ථිකයක්, ලබා,...","[වෝල්, වීදිය, නැවතත්, ධනාත්මක, ආර්ථිකයක්, ලබා,..."
1,3,කාර්ලයිල් වාණිජ අභ්‍යවකාශය දෙස බලයි (රොයිටර්) ...,"[කාර්ලයිල්, වාණිජ, අභ්යවකාශය, දෙස, බලයි, රොයිට...","[කාර්ලයිල්, වාණිජ, අභ්යවකාශය, දෙස, බලයි, රොයිට..."
2,3,තෙල් සහ ආර්ථිකය රොයිටර් - බොරතෙල් මිල ඉහළ යාම ...,"[තෙල්, සහ, ආර්ථිකය, රොයිටර්, බොරතෙල්, මිල, ඉහළ...","[තෙල්, සහ, ආර්ථිකය, රොයිටර්, බොරතෙල්, මිල, ඉහළ..."
3,3,ඉරාකය ප්‍රධාන දකුණු නල මාර්ගයෙන් (රොයිටර්) තෙල...,"[ඉරාකය, ප්රධාන, දකුණු, නල, මාර්ගයෙන්, රොයිටර්,...","[ඉරාකය, ප්රධාන, දකුණු, නල, මාර්ගයෙන්, රොයිටර්,..."
4,3,එක්සත් ජනපද ආර්ථිකයට (AFP) නව තර්ජනයක් එල්ල කර...,"[එක්සත්, ජනපද, ආර්ථිකයට, නව, තර්ජනයක්, එල්ල, ක...","[එක්සත්, ජනපද, ආර්ථිකයට, නව, තර්ජනයක්, එල්ල, ක..."


## Stemming

In [21]:
stem_dict = open(stem_dictionary, "r", encoding='utf-8')
stem_dict = map(lambda s: s.strip(), stem_dict)

stem_dictionary = {}

for s in stem_dict:
    s = s.split("\t")
    stem_dictionary[s[0]] = s[1]

In [22]:
def stemming_words(tokens):
    for k, v in enumerate(tokens):
        tokens[k] = stem_dictionary.get(v, v)
    tokens = " ".join([c for c in tokens if c not in string.punctuation])
    return tokens

In [23]:
df['Stemming Words (Paragraph)'] = df['Stopwords Removed (Paragraph)'].apply(lambda x: stemming_words(x))
df.head()

,Class Index,Paragraph,Cleaned Paragraph,Stopwords Removed (Paragraph),Stemming Words (Paragraph)
0,3,වෝල් වීදිය නැවතත් ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය...,"[වෝල්, වීදිය, නැවතත්, ධනාත්මක, ආර්ථිකයක්, ලබා,...","[වෝල්, වීදිය, නැවත, ධනාත්මක, ආර්ථිකයක්, ලබා, ග...",වෝල් වීදිය නැවත ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය න...
1,3,කාර්ලයිල් වාණිජ අභ්‍යවකාශය දෙස බලයි (රොයිටර්) ...,"[කාර්ලයිල්, වාණිජ, අභ්යවකාශය, දෙස, බලයි, රොයිට...","[කාර්ලයිල්, වාණිජ, අභ්යවකාශය, දෙස, බලයි, රොයිට...",කාර්ලයිල් වාණිජ අභ්යවකාශය දෙස බලයි රොයිටර් රොය...
2,3,තෙල් සහ ආර්ථිකය රොයිටර් - බොරතෙල් මිල ඉහළ යාම ...,"[තෙල්, සහ, ආර්ථිකය, රොයිටර්, බොරතෙල්, මිල, ඉහළ...","[තෙල්, සහ, ආර්ථිකය, රොයිටර්, බොරතෙල්, මිල, ඉහළ...",තෙල් සහ ආර්ථිකය රොයිටර් බොරතෙල් මිල ඉහළ යාම සහ...
3,3,ඉරාකය ප්‍රධාන දකුණු නල මාර්ගයෙන් (රොයිටර්) තෙල...,"[ඉරාකය, ප්රධාන, දකුණු, නල, මාර්ගයෙන්, රොයිටර්,...","[ඉරාක, ප්රධාන, දකුණු, නල, මාර්ගයෙන්, රොයිටර්, ...",ඉරාක ප්රධාන දකුණු නල මාර්ගයෙන් රොයිටර් තෙල් අප...
4,3,එක්සත් ජනපද ආර්ථිකයට (AFP) නව තර්ජනයක් එල්ල කර...,"[එක්සත්, ජනපද, ආර්ථිකයට, නව, තර්ජනයක්, එල්ල, ක...","[එක්සත, ජනපද, ආර්ථිකයට, නව, තර්ජනයක්, එල්ල, කර...",එක්සත ජනපද ආර්ථිකයට නව තර්ජනයක් එල්ල කරමින් තෙ...


In [24]:
df.drop(df.columns[[1, 2, 3]], axis=1, inplace=True)
df.head()

,Class Index,Stemming Words (Paragraph)
0,3,වෝල් වීදිය නැවත ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය න...
1,3,කාර්ලයිල් වාණිජ අභ්යවකාශය දෙස බලයි රොයිටර් රොය...
2,3,තෙල් සහ ආර්ථිකය රොයිටර් බොරතෙල් මිල ඉහළ යාම සහ...
3,3,ඉරාක ප්රධාන දකුණු නල මාර්ගයෙන් රොයිටර් තෙල් අප...
4,3,එක්සත ජනපද ආර්ථිකයට නව තර්ජනයක් එල්ල කරමින් තෙ...


In [25]:
df = df.rename(columns={'Class Index': 'Labels', 'Stemming Words (Paragraph)': 'Paragraph'})

In [26]:
df.head()

,Labels,Paragraph
0,3,වෝල් වීදිය නැවත ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය න...
1,3,කාර්ලයිල් වාණිජ අභ්යවකාශය දෙස බලයි රොයිටර් රොය...
2,3,තෙල් සහ ආර්ථිකය රොයිටර් බොරතෙල් මිල ඉහළ යාම සහ...
3,3,ඉරාක ප්රධාන දකුණු නල මාර්ගයෙන් රොයිටර් තෙල් අප...
4,3,එක්සත ජනපද ආර්ථිකයට නව තර්ජනයක් එල්ල කරමින් තෙ...


## Saving new preprocessed dataframe to CSV

In [27]:
df.to_csv('preprocessed_data.csv', index=False)

In [28]:
preprocessed_df = '/kaggle/input/preprocessed-data/preprocessed_data.csv'

In [29]:
df = pd.read_csv(preprocessed_df)
df.head()

,Labels,Paragraph
0,3,වෝල් වීදිය නැවත ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය න...
1,3,කාර්ලයිල් වාණිජ අභ්යවකාශය දෙස බලයි රොයිටර් රොය...
2,3,තෙල් සහ ආර්ථිකය රොයිටර් බොරතෙල් මිල ඉහළ යාම සහ...
3,3,ඉරාක ප්රධාන දකුණු නල මාර්ගයෙන් රොයිටර් තෙල් අප...
4,3,එක්සත ජනපද ආර්ථිකයට නව තර්ජනයක් එල්ල කරමින් තෙ...
